In [ ]:
import pandas as pd
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers.core import Dense,Dropout,Activation,Flatten
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN
import jieba

In [613]:
data_train=pd.read_csv(r'E:\baidumatch\sentiment\data\data_train.csv',encoding='gbk',delimiter="\t",header=None,index_col=0)
data_test=pd.read_csv(r'E:\baidumatch\sentiment\data\data_test.csv',encoding='gbk',delimiter='\t',header=None,index_col=0)
data_train

,1,2,3
0,,,
1,食品餐饮,买这套系统本来是用来做我们公司的公众号第三方平台代运营的，没想到还有app，而且每个都很方便...,2
2,食品餐饮,烤鸭还是不错的，别的菜没什么特殊的,1
3,食品餐饮,使用说明看不懂！不会用，很多操作没详细标明！,0
4,食品餐饮,越来越不好了，菜品也少了，服务也不及时。,0
5,食品餐饮,是在是不知道该吃什么好、就来了,1
6,食品餐饮,跟这家公司合作过一次，系统功能很强大，运行很流畅，很不错的。。,2
7,食品餐饮,点的中辣，但是比平时在店里吃的要辣很多，可能是叫外卖会比炒好直接吃花的时间长，辣味都出来了的...,2
8,食品餐饮,的确是专业，用心做，出品方面都给好评,2
9,食品餐饮,还不错，下次有机会还来哈哈哈啊哈哈哈还,2


In [612]:

data_train.columns=['a','b','c']
#data_test

In [ ]:
data_train.iloc[:,0]
dict={}
for i in data_train.iloc[:,0]:
    if i in dict:
        dict[i]+=1
    else:
        dict[i]=1
print(dict)

In [ ]:
#industry_dict={0:'食品餐饮',1:'医疗服务',2:'金融服务',3:'旅游住宿',4:'物流快递'}
industry_dict={'食品餐饮':0,'医疗服务':1,'金融服务':2,'旅游住宿':3,'物流快递':4}

In [ ]:
data_train.iloc[:,0]=data_train.iloc[:,0].map(industry_dict)#注意：不要写成data_train.iloc[:,0].map(lambda x:industry_dict),这样就将每一行都映射为industry_dict了

In [ ]:
data_train

In [ ]:
pinglun=data_train['b'].values.tolist()

In [ ]:
pinglun

In [ ]:
contents_cut=[]
for i in pinglun:
    sentence_cut=jieba.lcut(str(i))
    if sentence_cut!='\r\n': #有内容且不是换行符
        contents_cut.append(sentence_cut)

In [ ]:
len(contents_cut)
contents_cut

In [ ]:
global stopwords
stopwords=pd.read_csv(r'F:\tang_data\text_analy\stopwords.txt',index_col=False,sep='\t',quoting=3,names=['stopword'],encoding='utf-8')
#stopwords=pd.read_csv(r'F:\tang_data\text_analy\stopwords.txt',index_col=False,sep="\t",quoting=3,names=['stopword'], encoding='utf-8')
print(stopwords[:3])

In [ ]:
def drop_stopwords(contents,stopwords):
    contents_clean=[]
    all_clean_words=[]
    for line in contents:
        line_clean=[]
        for i in line:
            if i in stopwords or i=='[' or i==']':
                continue
            line_clean.append(i)
            all_clean_words.append(i)
            #line_clean_str=' '.join(line_clean_str)
        contents_clean.append(line_clean)
    return contents_clean,all_clean_words
stopwords=stopwords.stopword.values.tolist()
contents_clean,all_clean_words=drop_stopwords(contents_cut,stopwords)  #all_clean_words主要用来词云显示，对于算法模型无用
df_contents_clean=pd.DataFrame({'contents_cut_clean':contents_clean})
df_contents_clean.head()

In [ ]:
df_contents_clean.index = range(1,len(df_contents_clean) + 1)

In [ ]:
df_contents_clean

In [ ]:
data_train.b=df_contents_clean.contents_cut_clean

In [ ]:
data_train

In [614]:
data_train.b=data_train.b.map(lambda x:' '.join(x))

AttributeError: 'DataFrame' object has no attribute 'b'

In [ ]:
data_train

In [ ]:
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

In [ ]:
train_0=data_train.loc[data_train['a'] == 0]

In [ ]:
train_0

In [ ]:
token=Tokenizer(num_words=2000)
token.fit_on_texts(train_0.b)

In [ ]:
print(token.document_count)

In [ ]:
print(token.word_index)

In [ ]:
train_0_seq=token.texts_to_sequences(train_0.b)

In [ ]:
print(train_0_seq)

In [ ]:
x_0=sequence.pad_sequences(train_0_seq,15)

In [ ]:
print(x_0)

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense,Dropout,Activation,Flatten
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN

In [ ]:
model=Sequential()
model.add(Embedding(output_dim=32,input_dim=2000,input_length=15))
model.add(Dropout(0.2))


In [ ]:
#model.add(Flatten())
model.add(SimpleRNN(units=16))
model.add(Dense(units=250,activation='relu'))
model.add(Dropout(0.35))


In [ ]:
model.add(Dense(units=3,activation='softmax'))

In [ ]:
'''
import numpy as np
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
class Metrics(Callback):
  def on_train_begin(self, logs={}):
    self.val_f1s = []
    self.val_recalls = []
    self.val_precisions = []

  def on_epoch_end(self, epoch, logs={}):
    val_predict=(np.asarray(self.model.predict(self.model.validation_data[0]))).round()
    val_targ = self.model.validation_data[1]
    _val_f1 = f1_score(val_targ, val_predict)
    _val_recall = recall_score(val_targ, val_predict)
    _val_precision = precision_score(va_targ, val)
    self.val_f1s.append(_val_f1)
    self.val_recalls.append(_val_recall)
    self.val_precisions.append(_val_precision)
    print('-val_f1: %.4f --val_precision: %.4f --val_recall: %.4f'%(_val_f1, _val_precision, _val_recall))
    return

metrics = Metrics()

'''

In [ ]:
adam=keras.optimizers.Adamax(lr=0.008)

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['categorical_accuracy'])
#help(model.compile)

In [ ]:
model.summary()

In [ ]:
y_0=train_0.c

In [ ]:
from keras.utils import np_utils
y_0_onehot=np_utils.to_categorical(y_0)
y_0_onehot

In [ ]:
train_history=model.fit(x_0,y_0_onehot,batch_size=100,epochs=60,verbose=2,validation_split=0.2)

In [ ]:
import keras
help(keras.optimizers.Adamax)

In [ ]:
from keras.utils import np_utils
train_1=data_train.loc[data_train['a'] == 1]
train_1_seq=token.texts_to_sequences(train_1.b)
x_1=sequence.pad_sequences(train_1_seq,15)
y_1=train_1.c
y_1_onehot=np_utils.to_categorical(y_1)

train_2=data_train.loc[data_train['a'] == 2]
train_2_seq=token.texts_to_sequences(train_2.b)
x_2=sequence.pad_sequences(train_2_seq,15)
y_2=train_2.c
y_2_onehot=np_utils.to_categorical(y_2)

train_3=data_train.loc[data_train['a'] == 3]
train_3_seq=token.texts_to_sequences(train_3.b)
x_3=sequence.pad_sequences(train_3_seq,15)
y_3=train_3.c
y_3_onehot=np_utils.to_categorical(y_3)

train_4=data_train.loc[data_train['a'] == 4]
train_4_seq=token.texts_to_sequences(train_4.b)
x_4=sequence.pad_sequences(train_4_seq,15)
y_4=train_4.c
y_4_onehot=np_utils.to_categorical(y_4)

In [ ]:
train_history=model.fit(x_1,y_1_onehot,batch_size=100,epochs=60,verbose=2,validation_split=0.2)

In [ ]:
train_history=model.fit(x_4,y_4_onehot,batch_size=100,epochs=60,verbose=2,validation_split=0.2)